<a id="0"></a> <br>
Indice 
- [Librerias](#1)
- [Primera prueba google](#2)
- [Prueba Psn](#3)
- [Columnas de DF para webscrappear](#4)
- [Código webscrappeo sin necesidad de navegador abierto](#5)


<a id="1"></a> <br>
Librerias

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
from datetime import datetime
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup as bs
import requests
import re


In [2]:
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
# options.add_argument('--start-maximized') SOLO EN PC SOBREMESA si fuera necesario.

[Volver al índice](#0)

<a id="2"></a> <br>
Primera prueba google

In [11]:
driver = webdriver.Chrome(service=service, options=options)

driver.get("http://www.google.es")

time.sleep(3)

rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/span/div/div/div/div[3]/div[1]/button[1]/div')
rechazar_cookies.click()

time.sleep(3)

driver.quit()

[Volver al índice](#0)

<a id="3"></a> <br>
Webscrappeo Psn

In [3]:
df_juegos = pd.DataFrame(columns=["Titulo","Precio_original","Precio_actual","Precio con mayor rebaja"])

In [4]:
df_juegos

,Titulo,Precio_original,Precio_actual,Precio con mayor rebaja


In [ ]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc...
# Reset df
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()

df_juegos = pd.DataFrame(columns=["Titulo","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio_original_con_PSN","Precio_actual_sin_PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)
# element = WebDriverWait(driver, 10).until(EC.element_to_be_visible((By.ID, "card-51"))) intento de que sea mas funcional

driver.get("https://store.playstation.com/")


lista_tiempo = [1.0,1.5,2.0] #2.5,2.6,2.7,2.8,2.9,3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

# Obtenemos número de juegos para scrapear
time.sleep(random.choice(lista_tiempo))
page_source = driver.page_source
soup = bs(page_source, 'lxml')
# Mostrar un numero de juegos limitado
numero_juegos = 10
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego

cont = 0
while cont <= 24:
    cont += 1
    time.sleep(random.choice(lista_tiempo))
    select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
    select_game.click()

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')

    # Variable por juego
    
    # Titulo
    try:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
    except:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
    
    # Precio original
    try:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
    except:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
    
    # Precio actual 
    try:
        precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
    except:
        precio_actual_sn_psn = precio_original_sn_psn
    
    # Plataforma
    try:
        plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
    except:
        plataforma = "No hay información"
    
    # Genero
    try:
        genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
    except:
        genero = "No hay información"
    
    # Compañia
    try:
        compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
    except:
        compania = "No hay información"
    
    # Lanzamiento
    try:  
        lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
    except:
        lanzamiento = "No hay información"
    
    # Idiomas
    try:
        idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
    except:
        idiomas = "No hay información"
    # Nº Calificaciones
    try:
        num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
    except:
        num_calificaciones = "No hay información"
    # Calificación PSN
    
    try:
        calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
    except:
        calificacion = "No hay información"
    try:
        calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
    except:
        calificacion_1 = "No hay información"
    try:
        calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
    except:
        calificacion_2 = "No hay información"
    try:
        calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
    except:
        calificacion_3 = "No hay información"
    try:
        calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
    except:
        calificacion_4 = "No hay información"
    try:
        calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
    except:
        calificacion_5 = "No hay información"
    
    # precio_con_mayor_rebaja = "No hay información"
    
    df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                    "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                    "Calificación 5 estrellas":calificacion_5,
                                    "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                    "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                    "Precio original sin PSN":precio_original_sn_psn,"Precio_actual sin PSN":precio_actual_sn_psn}
                                    # "Precio con mayor rebaja":precio_con_mayor_rebaja 
    # time.sleep(random.choice(lista_tiempo))

    driver.back();

    time.sleep(random.choice(lista_tiempo))
    #Comprobar error aquí
    if cont == 24:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()      
        cont = 0
    elif numero_juegos == len(df_juegos):
        break
    else:
        continue           
driver.quit()


1 minuto aprox cada juego.

Prueba identificacion numero de juegos.

In [11]:
df_juegos.head()

,Titulo,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio_original_con_PSN,Precio_actual_sin_PSN
0,01 Deadliest Zone Catch — Boat Crab & Fishing ...,PS4,"Acción, Simulación, Únicos",Midnight Works SRL,29/8/2023,Inglés,1.9,100 calificaciones,10 %,5 %,15 %,5 %,65 %,"14,99 €","8,99 €"
1,0 Degrees,PS5,"Acción, Puzzle",eastasiasoft,31/12/2021,Inglés,3.92,78 calificaciones,53 %,9 %,24 %,6 %,8 %,"4,99 €","14,99 €"
2,0 Rally Desert Race: Offroad Dirt Simulator,PS4,"Acción, Conducción/Carreras, Simulación",Midnight Works SRL,1/9/2023,Inglés,2.68,88 calificaciones,27 %,8 %,13 %,10 %,42 %,"14,99 €","8,99 €"
3,1001 Spikes,PS4,"Acción, Acción, Arcade",Nicalis Inc.,7/10/2015,Inglés,4.43,892 calificaciones,67 %,20 %,6 %,3 %,4 %,"14,99 €","14,99 €"
4,100ft Robot Golf,PS4,"Grupo, Deporte",NO GOBLIN LLC,18/9/2017,Inglés,3.42,285 calificaciones,38 %,17 %,13 %,12 %,20 %,"9,99 €","14,99 €"


[Volver al índice](#0)

<a id="4"></a> <br>
Columnas de DF para webscrappear

In [42]:
soup

<html><head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body></html>

In [ ]:
# Titulo
try:
    print(soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text())
except:
    print(soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text())

In [ ]:
# Precio Original sin PSN
soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()


In [31]:
#Precio Actual sin PSN
try:
    print(soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text())
except: 
    print(soup.find("span",class_="psw-t-title-m").get_text())

4,99 €


In [ ]:
# Precio original con PSN
try:
    print(soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text())
except: 
    soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()

In [ ]:
#Precio Actual con PSN
try:
    print(soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text())
except: 
    try:
        print(soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text())
    except: 
        print(soup.find("span",class_="psw-t-title-m").get_text())

In [29]:
# Plataforma
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()


'PS4'

In [21]:
# Genero
soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()

'Aventura, Puzzle'

In [23]:
# Compañia
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()

'eastasiasoft'

In [24]:
# Lanzamiento
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()

'10/11/2021'

In [26]:
# Idiomas
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()


'Alemán, Chino (Tradicional), Español, Francés (Francia), Inglés, Portugués (Brasil)'

In [33]:
# Calificación PSN
soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()


'2.57'

In [34]:
# Nº Calificaciones
soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()

'37 calificaciones'

In [35]:
# % 5 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()

'27 %'

In [36]:
# % 4 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()

'5 %'

In [37]:
# % 3 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()

'11 %'

In [38]:
# % 2 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()

'11 %'

In [39]:
# % 1 estrella
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()

'46 %'

In [56]:
# Precio al día
fecha_webs = datetime.now()
fecha_webs = datetime.isoformat(fecha_webs)

[Volver al índice](#0)

<a id="5"></a> <br>
Código webscrappeo sin necesidad de navegador abierto

In [ ]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc...
# Reset df
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")


df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)


driver.get("https://store.playstation.com/")


lista_tiempo = [1.9,2.0,2.1,2.2]

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

# Obtenemos número de juegos para scrapear
time.sleep(random.choice(lista_tiempo))
page_source = driver.page_source
soup = bs(page_source, 'lxml')
# Mostrar un numero de juegos limitado
numero_juegos = 1000
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego

cont = 0
while cont <= 24:
    cont += 1
    time.sleep(random.choice(lista_tiempo))
    select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
    select_game.click()

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.
    page_source = driver.page_source
    soup = bs(page_source, 'html')

    # Variable por juego
    
    # Titulo
    try:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
    except:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
    
    # Día y hora de webscrappeo
    
        fecha_webs = datetime.now()
        fecha_webs = datetime.isoformat(fecha_webs)
        
    # Precio original sin PSN
    try:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
    except:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
        
    # Precio original con PSN
    try:
        precio_original_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'}).get_text()
    except: 
        try:
            precio_original_cn_psn = soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text()
        except:
            precio_original_cn_psn = precio_original_sn_psn
    # Precio actual sin PSN
    try:
        precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
    except:
        precio_actual_sn_psn = precio_original_sn_psn
    
    #Precio Actual con PSN
    try:
        precio_actual_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text()
    except: 
        try:
            precio_actual_cn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
        except: 
            precio_actual_cn_psn = precio_actual_sn_psn
    
    # Plataforma
    try:
        plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
    except:
        plataforma = "No hay información"
    
    # Genero
    try:
        genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
    except:
        genero = "No hay información"
    
    # Compañia
    try:
        compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
    except:
        compania = "No hay información"
    
    # Lanzamiento
    try:  
        lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
    except:
        lanzamiento = "No hay información"
    
    # Idiomas
    try:
        idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
    except:
        idiomas = "No hay información"
    # Nº Calificaciones
    try:
        num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
    except:
        num_calificaciones = "No hay información"
    # Calificación PSN
    
    try:
        calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
    except:
        calificacion = "No hay información"
    try:
        calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
    except:
        calificacion_1 = "No hay información"
    try:
        calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
    except:
        calificacion_2 = "No hay información"
    try:
        calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
    except:
        calificacion_3 = "No hay información"
    try:
        calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
    except:
        calificacion_4 = "No hay información"
    try:
        calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
    except:
        calificacion_5 = "No hay información"
    
    # precio_con_mayor_rebaja = "No hay información"
    
    df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                    "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                    "Calificación 5 estrellas":calificacion_5,
                                    "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                    "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                    "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn}
                                    # "Precio con mayor rebaja":precio_con_mayor_rebaja 
    # time.sleep(random.choice(lista_tiempo))

    driver.back();

    time.sleep(random.choice(lista_tiempo))
    #Comprobar error aquí
    if cont == 24:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()
        time.sleep(random.choice(lista_tiempo))      
        cont = 0
    elif numero_juegos == len(df_juegos):
        break
    else:
        continue           
driver.quit()

Funciones

In [3]:
def carga_pagina_inicial():
    from selenium.common.exceptions import TimeoutException
    from fake_useragent import UserAgent
    "fake_user_agent"
    ua = UserAgent()
    timeout = 10
    
    # lista_tiempo = [3,3.1,3.2]
    #rechazamos cookies

    try:
        butt_coo = EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img'))
        WebDriverWait(driver, timeout).until(butt_coo)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
    rechazar_cookies.click()

    #vamos a la pestaña de explora

    try:
        expl_butt = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a'))
        WebDriverWait(driver, timeout).until(expl_butt)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
    explora.click()


    #plat

    try:
        sort_butt = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button'))
        WebDriverWait(driver, timeout).until(sort_butt)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")


    sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
    sort_plat.click()


    #ordenamos

    try:
        sort_g = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button'))
        WebDriverWait(driver, timeout).until(sort_g)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")


    sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
    sort_gam.click()

    #AZ


    try:
        sort_az_ = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]'))
        WebDriverWait(driver, timeout).until(sort_az_)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
    sort_az.click()

    # Obtenemos número de juegos para scrapear

    # time.sleep(random.choice(lista_tiempo))

    driver.implicitly_wait(10)
    return
            
def pagina_concreta_carga(pagina):
    pag = 1
    while pag != pagina:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()
        pag += 1

In [10]:

### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc... (Prueba con timeouts en vez de random choice de tiempos)
# Reset df
from selenium.common.exceptions import TimeoutException
from fake_useragent import UserAgent
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
timeout = 10

df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://store.playstation.com/")


lista_tiempo = [3,3.1,3.2]

carga_pagina_inicial()
# Mostrar un numero de juegos limitado
numero_juegos = 1000
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego
page = 1
cont = 0 # Establecemos el primer juego que estará en cont = 1, pero lo establecemos en 0 para iniciarlo
while cont <= 24:
    try:
        cont += 1
        try:
            sel_game = EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a'))
            WebDriverWait(driver, timeout).until(sel_game)
        except TimeoutException:
            print(f"Timed out waiting for game to appear, game number{cont}")
        
        select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
        select_game.click()

        
        # Variable por juego
        
        # Titulo
        
        # Info que nos descargamos de la pagina del juego.

        driver.implicitly_wait(10)
        try:  
            url = driver.current_url
            headers = {'User-Agent': ua.random}
            response = requests.get(url, headers=headers)
            soup = bs(response.text,'html')
        except Exception as e:
            print(f"Error al obtener la URL: error en el juego{cont}, página{page}")
            

        try: #Intentamos primero con un tipo de letra
            titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
        except: #Intentamos después con otro tipo de letra
            titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
        
        # Día y hora de webscrappeo
        
        fecha_webs = datetime.now()
        fecha_webs = datetime.isoformat(fecha_webs)
            
        # Precio original sin PSN
        try:
            precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
        except:
            precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
            
        # Precio original con PSN
        try:
            precio_original_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'}).get_text()
        except: 
            try:
                precio_original_cn_psn = soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text()
            except:
                precio_original_cn_psn = precio_original_sn_psn
        # Precio actual sin PSN
        try:
            precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
        except:
            precio_actual_sn_psn = precio_original_sn_psn
        
        #Precio Actual con PSN
        try:
            precio_actual_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text()
        except: 
            try:
                precio_actual_cn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
            except: 
                precio_actual_cn_psn = precio_actual_sn_psn
        
        # Plataforma
        try:
            plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
        except:
            plataforma = "No hay información"
        
        # Genero
        try:
            genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
        except:
            genero = "No hay información"
        
        # Compañia
        try:
            compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
        except:
            compania = "No hay información"
        
        # Lanzamiento
        try:  
            lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
        except:
            lanzamiento = "No hay información"
        
        # Idiomas
        try:
            idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
        except:
            idiomas = "No hay información"
        # Nº Calificaciones
        try:
            num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
        except:
            num_calificaciones = "No hay información"
        # Calificación PSN
        
        try:
            calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
        except:
            calificacion = "No hay información"
        try:
            calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
        except:
            calificacion_1 = "No hay información"
        try:
            calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
        except:
            calificacion_2 = "No hay información"
        try:
            calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
        except:
            calificacion_3 = "No hay información"
        try:
            calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
        except:
            calificacion_4 = "No hay información"
        try:
            calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
        except:
            calificacion_5 = "No hay información"
        
        # precio_con_mayor_rebaja = "No hay información"
        
        df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                        "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                        "Calificación 5 estrellas":calificacion_5,
                                        "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                        "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                        "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn}
                                        # "Precio con mayor rebaja":precio_con_mayor_rebaja 
        
        time.sleep(random.choice(lista_tiempo))
        try:  
            driver.back();
        except:
            print(f"Timed out waiting for game to appear, game number{cont}, reinicio captura")
            driver.quit()
            driver = webdriver.Chrome(service=service, options=options)
            service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
            options = webdriver.ChromeOptions()
            driver.get("https://store.playstation.com/")
            carga_pagina_inicial()
            pagina_concreta_carga(page)
            continue

        
        #Comprobar error aquí
        if cont == 24:
            next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
            next_page.click()
            page += 1      
            cont = 0
            time.sleep(random.choice(lista_tiempo))
        elif len(df_juegos) == 300:
            # Volvemos a hacer la carga completa de la pagina
            driver.quit()
            time.sleep(5)
            driver = webdriver.Chrome(service=service, options=options)
            service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
            options = webdriver.ChromeOptions()
            driver.get("https://store.playstation.com/")
            carga_pagina_inicial()
            pagina_concreta_carga(page)
            print("Número de juegos completados de webscrapear", str(len(df_juegos)))
            continue
        elif numero_juegos == len(df_juegos):
            break
        else:
            continue
    except:
        # Volvemos a hacer la carga completa de la pagina
        driver.quit()
        driver = webdriver.Chrome(service=service, options=options)
        service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
        options = webdriver.ChromeOptions()
        driver.get("https://store.playstation.com/")
        carga_pagina_inicial()
        pagina_concreta_carga(page)
                  
driver.quit()

MEJORAMOS EL FLUJO PARA LOS TIEMPOS DE ESPERA

In [6]:
# MEJORANDO EL SCRAPEO IDENTIFICANDO EL SOURCE Y OBTENIENDO TODOS LOS DATOS DE ELLA

### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc... (Prueba con timeouts en vez de random choice de tiempos)
# Reset df
from selenium.common.exceptions import TimeoutException
from fake_useragent import UserAgent
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")
timeout = 10

df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)

link_inicial = "https://store.playstation.com/"
driver.get(link_inicial)


lista_tiempo = [3,3.1,3.2]

carga_pagina_inicial()
# Mostrar un numero de juegos limitado
numero_juegos = 20
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego
page = 1
cont = -1 # Establecemos el primer juego que estará en cont = 1, pero lo establecemos en 0 para iniciarlo
while cont <= 24:
    try:
        cont += 1
        try:
            sel_game = EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a'))
            WebDriverWait(driver, timeout).until(sel_game)
        except TimeoutException:
            print(f"Timed out waiting for game to appear, game number {cont}")
        
        # select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
        # select_game.click()   
        
        # Variable por juego
        # Titulo
        # Info que nos descargamos de la pagina del juego.

        driver.implicitly_wait(10)
        try:
            url = driver.current_url  
            headers = {'User-Agent': ua.random}
            response = requests.get(url, headers=headers)
            soup_pagina_entera = bs(response.text,'html.parser')
            url_game = soup_pagina_entera.select_one(f'[data-qa="ems-sdk-grid#productTile{cont}"] a')
            href_valor = url_game.get('href')
            link_juego = link_inicial + href_valor
            
            headers = {'User-Agent': ua.random}
            response = requests.get(link_juego, headers=headers)
            soup = bs(response.text,'html')
            print(link_juego)
        except Exception as e:
            print(f"Error al obtener la URL: error en el juego{cont}, página{page}")
            
        # Aquí vamos a coger el soup de cada url de cada juego para obtener la info
        
        
        
        try: #Intentamos primero con un tipo de letra
            titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
        except: #Intentamos después con otro tipo de letra
            titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
        
        # Día y hora de webscrappeo
        
        fecha_webs = datetime.now()
        fecha_webs = datetime.isoformat(fecha_webs)
            
        # Precio original sin PSN
        try:
            precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
        except:
            precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
            
        # Precio original con PSN
        try:
            precio_original_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'}).get_text()
        except: 
            try:
                precio_original_cn_psn = soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text()
            except:
                precio_original_cn_psn = precio_original_sn_psn
        # Precio actual sin PSN
        try:
            precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
        except:
            precio_actual_sn_psn = precio_original_sn_psn
        
        #Precio Actual con PSN
        try:
            precio_actual_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text()
        except: 
            try:
                precio_actual_cn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
            except: 
                precio_actual_cn_psn = precio_actual_sn_psn
        
        # Plataforma
        try:
            plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
        except:
            plataforma = "No hay información"
        
        # Genero
        try:
            genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
        except:
            genero = "No hay información"
        
        # Compañia
        try:
            compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
        except:
            compania = "No hay información"
        
        # Lanzamiento
        try:  
            lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
        except:
            lanzamiento = "No hay información"
        
        # Idiomas
        try:
            idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
        except:
            idiomas = "No hay información"
        # Nº Calificaciones
        try:
            num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
        except:
            num_calificaciones = "No hay información"
        # Calificación PSN
        
        try:
            calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
        except:
            calificacion = "No hay información"
        try:
            calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
        except:
            calificacion_1 = "No hay información"
        try:
            calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
        except:
            calificacion_2 = "No hay información"
        try:
            calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
        except:
            calificacion_3 = "No hay información"
        try:
            calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
        except:
            calificacion_4 = "No hay información"
        try:
            calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
        except:
            calificacion_5 = "No hay información"
        
        # precio_con_mayor_rebaja = "No hay información"
        
        df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                        "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                        "Calificación 5 estrellas":calificacion_5,
                                        "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                        "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                        "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn}
                                        # "Precio con mayor rebaja":precio_con_mayor_rebaja 
        
        # time.sleep(random.choice(lista_tiempo))
    
        #Comprobar error aquí
        if cont == 23:
            next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
            next_page.click()
            page += 1      
            cont = -1
            time.sleep(random.choice(lista_tiempo))
        elif len(df_juegos) == 300:
            # Volvemos a hacer la carga completa de la pagina
            driver.quit()
            time.sleep(5)
            driver = webdriver.Chrome(service=service, options=options)
            service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
            options = webdriver.ChromeOptions()
            options.add_argument("--headless=new")
            driver.get("https://store.playstation.com/")
            carga_pagina_inicial()
            pagina_concreta_carga(page)
            print("Número de juegos completados de webscrapear", str(len(df_juegos)))
            continue
        elif numero_juegos == len(df_juegos):
            break
        else:
            continue
    except:
        # Volvemos a hacer la carga completa de la pagina
        print("error carga")
        driver.quit()
        driver = webdriver.Chrome(service=service, options=options)
        service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
        options = webdriver.ChromeOptions()
        options.add_argument("--headless=new")
        driver.get("https://store.playstation.com/")
        carga_pagina_inicial()
        pagina_concreta_carga(page)
                  
driver.quit()

Timed out waiting for game to appear, game number 0
https://store.playstation.com//es-es/concept/10007176
https://store.playstation.com//es-es/concept/228748
https://store.playstation.com//es-es/concept/10001130
https://store.playstation.com//es-es/concept/10007788
https://store.playstation.com//es-es/concept/201930
https://store.playstation.com//es-es/concept/10001609
https://store.playstation.com//es-es/concept/10000896
https://store.playstation.com//es-es/concept/10002427
https://store.playstation.com//es-es/concept/10006270
https://store.playstation.com//es-es/concept/10001555
https://store.playstation.com//es-es/concept/229066
https://store.playstation.com//es-es/concept/209441
https://store.playstation.com//es-es/concept/230079
https://store.playstation.com//es-es/concept/232447
https://store.playstation.com//es-es/concept/10003999
https://store.playstation.com//es-es/concept/10002313
https://store.playstation.com//es-es/concept/234567
https://store.playstation.com//es-es/concept

In [38]:
# Prueba web
from fake_useragent import UserAgent
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
timeout = 10

df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)

link_inicial = "https://store.playstation.com/"

driver.get(link_inicial)


lista_tiempo = [3,3.1,3.2]

carga_pagina_inicial()
cont = 0
url = driver.current_url
headers = {'User-Agent': ua.random}
response = requests.get(url, headers=headers)
soup_pagina_entera = bs(response.text,'html.parser')
url_game = soup_pagina_entera.select_one(f'[data-qa="ems-sdk-grid#productTile{cont}"] a')
href_valor = url_game.get('href')
link_juego = link_inicial + href_valor
print(link_juego)

driver.quit()

https://store.playstation.com//es-es/concept/10007176


In [16]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 0 to 69
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Titulo                    70 non-null     object
 1   Día y hora                70 non-null     object
 2   Plataforma                70 non-null     object
 3   Genero                    70 non-null     object
 4   Compañia                  70 non-null     object
 5   Lanzamiento               70 non-null     object
 6   Idiomas                   70 non-null     object
 7   Calificación PSN          70 non-null     object
 8   Número de calificaciones  70 non-null     object
 9   Calificación 5 estrellas  70 non-null     object
 10  Calificación 4 estrellas  70 non-null     object
 11  Calificación 3 estrellas  70 non-null     object
 12  Calificación 2 estrellas  70 non-null     object
 13  Calificación 1 estrella   70 non-null     object
 14  Precio original sin PSN   70 non-

[Volver al índice](#0)

In [17]:
df_juegos

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN
0,EA SPORTS FC™ 24 Standard Edition para PS4 y PS5,2024-02-06T13:25:03.120675,"PS4, PS5",No hay información,EA Swiss Sarl,28/9/2023,No hay información,3.49,49.451 calificaciones,47 %,12 %,9 %,5 %,26 %,"79,99 €","27,99 €",,"27,99 €"
1,Fortnite,2024-02-06T13:25:05.564419,PS5,No hay información,EPIC GAMES,21/7/2017,"Alemán, Coreano, Español, Español (México), Fr...",4.32,6.606.267 calificaciones,71 %,10 %,8 %,1 %,10 %,Gratis,Gratis,Gratis,Gratis
2,Call of Duty®: Modern Warfare® III - Lote Mult...,2024-02-06T13:25:09.404810,"PS4, PS5",No hay información,Activision Blizzard Int'l BV,2/11/2023,"Alemán, Español, Francés (Francia), Inglés, It...",4.05,271.732 calificaciones,62 %,11 %,11 %,2 %,14 %,"79,99 €","55,99 €",,"55,99 €"
3,Edición Kobe Bryant de NBA 2K24 para PS5™,2024-02-06T13:25:12.789039,PS5,No hay información,2K,7/9/2023,"Alemán, Chino (Simplificado), Chino (Tradicion...",3.23,16.082 calificaciones,37 %,14 %,12 %,7 %,29 %,"79,99 €","23,99 €",,"23,99 €"
4,Grand Theft Auto Online (PlayStation®5),2024-02-06T13:25:16.464045,PS5,No hay información,Rockstar Games,14/3/2022,"Alemán, Chino (Simplificado), Chino (Tradicion...",4.52,670.462 calificaciones,79 %,8 %,5 %,1 %,7 %,"39,99 €","19,99 €",,"19,99 €"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,FAR CRY®6 Standard Edition PS4 & PS5,2024-02-06T13:29:10.752215,"PS4, PS5",No hay información,UBISOFT ENTERTAINMENT SA,6/10/2021,"Alemán, Español, Español (México), Francés (Fr...",4.05,41.006 calificaciones,61 %,13 %,9 %,3 %,13 %,"69,99 €","17,49 €","69,99 €",Incluido
66,Dead Island 2,2024-02-06T13:29:13.908302,"PS4, PS5",No hay información,Deep Silver,20/4/2023,"Alemán, Checo, Chino (Simplificado), Chino (Tr...",4.4,5.673 calificaciones,67 %,20 %,6 %,3 %,5 %,"69,99 €","41,99 €",,"41,99 €"
67,WORLD OF WARSHIPS: LEGENDS,2024-02-06T13:29:17.567571,PS5,No hay información,Wargaming Group Ltd,26/4/2021,"Alemán, Chino (Simplificado), Chino (Tradicion...",4.12,104.424 calificaciones,60 %,15 %,13 %,2 %,11 %,Gratis,Gratis,Gratis,Gratis
68,World of Tanks,2024-02-06T13:29:38.339143,PS5,No hay información,Wargaming Group Ltd,26/10/2021,"Alemán, Español, Francés (Francia), Inglés, Ja...",4.1,278.025 calificaciones,61 %,13 %,12 %,2 %,12 %,Gratis,Gratis,Gratis,Gratis


arreglar precios.

Creando base de datos en server elephant

In [1]:
import psycopg2

conn = psycopg2.connect(database="wgjqejjm",
                        host="manny.db.elephantsql.com",
                        user="wgjqejjm",
                        password="EeAurbeM3ZiVV3mq1zeKRkp8_eL_DZUB",
                        port="5432")

In [2]:
cursor = conn.cursor()

In [3]:
table_creation = '''
    CREATE TABLE info_juegos (
        game_id SERIAL PRIMARY KEY,
        game_name TEXT NOT NULL
    )
'''

In [4]:
cursor.execute(table_creation)

In [5]:
conn.commit()
cursor.close()
conn.close()